In [1]:
from dask.distributed import Client
client = Client()
# client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:49221 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 17.11 GB


In [ ]:
# <editor-fold desc=" ===== Import ======================================== ">
# from scripts.project_settings import *
from datetime import datetime
import numpy as np
import pandas as pd
from ta import *
import tpot
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer


# Change pandas display options to show full tables
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 25)
# </editor-fold>

# <editor-fold desc=" ===== Load data ===================================== ">
strategyFolder = 'swing'
source_path = '../../data/models_raw/swing/' + strategyFolder + '/'

csv_source_file = (
    'EURUSD_H1_20050101_20191026_swingV03_0200-0080-0040-0040.csv'
)

source_file = str(source_path + csv_source_file)

destination_path = '../../data/models_raw/' + strategyFolder + '/'
output_file = destination_path + csv_source_file.rsplit("_", 2)[0]

df = pd.read_csv(source_file, sep=",", encoding='utf-8', engine='c')

# # TODO 1 REMOVE THIS LINE !!!
# df = df.head(1000)

df = df.sort_values(
    by=[str('ID')], ascending=True, na_position='first'
)
df = df.drop_duplicates(subset='ID')
df = df.reset_index(drop=True)
df = df.round({
    # 'volume': 0,
    'open': 5,
    'high': 5,
    'low': 5,
    'close': 5
})
# </editor-fold>

# <editor-fold desc=" ===== Pre-process and split ========================= ">

# split variables and targets, and remove fields that can't be pre-processed
dfData = df.drop(['ID', 'timestamp', 'target'], axis=1)
dfTarget = df[['target']]

# Pre-process variables and convert to numpy arrays
qt = QuantileTransformer()
data = qt.fit_transform(dfData)
target = dfTarget.values

# Get number of rows in df to split data set
rowsdf, colsdf = data.shape
trainRows = round(rowsdf * 0.9)
testRows = round(rowsdf - trainRows)

# Split into training and test data sets
X_train = data[:trainRows,:]
X_test = data[testRows:,:]
y_train = target[:trainRows,:]
y_test = target[testRows:,:]

y_train = y_train.ravel()
y_test = y_test.ravel()

# delete original df
del df, data, target

print('===== data load and pre-process complete =====')
# </editor-fold>

In [ ]:
# <editor-fold desc=" ===== TPOT ========================================== ">

# TPOT config
tpot_config = {
    'sklearn.neural_network.MLPClassifier': {
    }
}

# TPOT classifier
tp = TPOTClassifier(
#     config_dict=tpot_config,  # None
#     memory='/Users/mbp13/OneDrive/GitHub/Trading-Predictions-Framework/models/swing/TPOT_caching_macOS_01',
    memory='C:/Users/xau/OneDrive/GitHub/Trading-Predictions-Framework/models/swing/TPOT_caching',
    warm_start=True,
#     template='Selector-Transformer-Classifier',
    template='Classifier',
    generations=100,  # 100
    population_size=10,  # 100
    cv=3,  # 5
    n_jobs=-1,  # -1
    max_time_mins=None,  # None
    max_eval_time_mins=2,  # 5
    random_state=None,  # None
    early_stop=None,  # None
    use_dask=True,  # False
    verbosity=2  # 0
)


In [ ]:
# fit data
tp.fit(X_train, y_train)

In [ ]:
# score model
print(tp.score(X_test, y_test))

In [ ]:
# export optimised classifier code
tp.export('swingV03_tpotClassifierPipeline.py')

# </editor-fold>